# DENOISER


## Dataset creation

Dataset: https://www.kaggle.com/datasets/hsankesara/flickr-image-dataset

In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import random

#Define the paths to the directories
raw_data_dir = 'rawdata'
output_dir = 'Data'
clean_dir = os.path.join(output_dir, 'clean')
noisy_dir = os.path.join(output_dir, 'noisy')

os.makedirs(clean_dir, exist_ok=True)
os.makedirs(noisy_dir, exist_ok=True)

In [2]:
def add_gaussian_noise(image, mean=0, sigma=25):
    row, col, ch = image.shape
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    noisy = image + gauss
    noisy = np.clip(noisy, 0, 255)
    return noisy.astype(np.uint8)

def add_salt_and_pepper_noise(image, prob=0.05):
    output = np.copy(image)
    #Salt mode
    salt_mask = np.random.random(image.shape) < prob/2
    output[salt_mask] = 255
    #Pepper mode
    pepper_mask = np.random.random(image.shape) < prob/2
    output[pepper_mask] = 0
    return output

def add_poisson_noise(image, scale=1.0):
    noisy = np.random.poisson(image * scale) / scale
    return np.clip(noisy, 0, 255).astype(np.uint8)

def add_speckle_noise(image, std=0.1):
    noise = np.random.normal(0, std, image.shape)
    noisy = image + image * noise
    return np.clip(noisy, 0, 255).astype(np.uint8)

#List of noise functions and their parameters
noise_functions = [
    (add_gaussian_noise, {}),
    (add_salt_and_pepper_noise, {}),
    (add_poisson_noise, {'scale': 1.0}),
    (add_speckle_noise, {})
]

In [3]:
#Process images
image_files = os.listdir(raw_data_dir)
target_size = (256, 256)

for idx, img_name in enumerate(tqdm(image_files)):
    if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue
        
    #Read image
    img_path = os.path.join(raw_data_dir, img_name)
    img = cv2.imread(img_path)
    if img is None:
        continue
        
    #Resize image
    img_resized = cv2.resize(img, target_size)
    
    #Save clean image
    clean_path = os.path.join(clean_dir, f'{idx:05d}.png')
    cv2.imwrite(clean_path, img_resized)
    
    #Select random noise function
    noise_func, params = random.choice(noise_functions)
    
    #Add noise to image
    noisy_img = noise_func(img_resized, **params)
    
    #Save noisy image
    noisy_path = os.path.join(noisy_dir, f'{idx:05d}.png')
    cv2.imwrite(noisy_path, noisy_img)

100%|██████████| 31785/31785 [07:59<00:00, 66.30it/s] 


In [4]:
#Check the number of images generated
print(f"Clean images generated: {len(os.listdir(clean_dir))}")
print(f"Noisy images generated: {len(os.listdir(noisy_dir))}")

Clean images generated: 31783
Noisy images generated: 31783
